# Flooding frequency maps

#### Create the flooding frequency maps for each saltmarsh

In [1]:
# Import libraries
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as cx
from matplotlib.colors import ListedColormap
from typing import Dict, List
import numpy as np
import glob
from tqdm.auto import tqdm
import re

c:\Users\AMarley.ERG\.conda\envs\geospatial-new\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# data paths
RAW_DATA = "../data/raw/"
CLEAN_DATA = "../data/clean/"
SLAMM_RESULTS = CLEAN_DATA + "slamm/"
OUTPUT = "../output/"

In [3]:
# Set naming convention for categories
inun_freq_key_list = [
    [0, 'Always',], 
    [1, 'At least every 30-days',], 
    [2, 'At least every 60-days'], 
    [3, 'At least every 90-days'], 
    [4, '10-year storm surge'], 
    [5, '100-year storm surge'], 
    [8, 'Protected by dikes']
]

inun_freq_key_df = pd.DataFrame(inun_freq_key_list, columns=['inun_freq_code', 'inun_freq_leg'])

In [4]:
# Create color theme
flood_colors = {
    'Always': '#0F2B5FFF',
    'At least every 30-days': '#5991C7FF', 
    'At least every 60-days': '#8EC1E7FF', 
    'At least every 90-days': '#B9DBF1FF', 
    '10-year storm surge': '#FDB958FF', 
    '100-year storm surge': '#FCE3A8FF', 
    'Protected by dikes': 'purple'
}

In [5]:
# set order of legend
order_leg = [
    'Always', 
    'At least every 30-days',
    'At least every 60-days',
    'At least every 90-days',
    '10-year storm surge',
    '100-year storm surge',
    'Protected by dikes'
]

In [6]:
# Import functions from util
from util import add_missing_cats
from util import create_categorical_cmap

In [18]:
# function to make the flooding frequency maps
def make_flood_map(slamm_file_path:str, leg_names:pd.DataFrame, color_dict, order_leg, output_path:str, cranberry_path:str):

    # read in file
    slamm_file = gpd.read_file(slamm_file_path)
    slamm_file = slamm_file.merge(leg_names, on='inun_freq_code', how='left')

    # fill in missing values
    slamm_file = add_missing_cats(slamm_file, order_leg, 'blank')

    # set the x and y lims for each marsh
    if 'ri2' in slamm_file_path:
        xleft = 0
        xright = 0
        yup = 0
    if 'ma2' in slamm_file_path:
        xleft = 0
        xright = 0
        yup = 0.001e6
    if 'cc1' in slamm_file_path:
        xleft = 0.0002e6
        xright = 0.00025e6
        yup = 0.001e6

    # set x and y limits
    xlim = (slamm_file.total_bounds[0] + xleft,  slamm_file.total_bounds[2] - xright)
    ylim = (slamm_file.total_bounds[1] + yup,  slamm_file.total_bounds[3])

    # Make map
    fig, ax = plt.subplots(figsize=(15, 10))
    slamm_file.plot(ax=ax, 
                column='inun_freq_leg', 
                cmap = create_categorical_cmap(color_dict, order_leg),
                alpha=0.5, 
                legend=True)
    cx.add_basemap(ax, source=cx.providers.CartoDB.Positron, crs = slamm_file.crs, zoom=14)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.set_xticks([])
    ax.set_yticks([])
    legend = ax.get_legend()
    handles = [None] * len(order_leg)
    for handle, label in zip(legend.legend_handles, legend.texts):
        text = label.get_text()
        handles[order_leg.index(text)] = handle
    ax.legend(handles, order_leg, title='Flooding Frequency')
    ax

    # Save map
    fig.savefig(output_path, dpi=350, bbox_inches="tight")

In [ ]:
# Make a flooding frequency map for each marsh and sea level rise and marsh scenario
FILE_PATH_PATTERN = re.compile(r"(cc1|ma2|ri2)\/.*(1|2)slr_.*(20|70|40)_(n|s).*\.geojson", re.IGNORECASE)
pbar = tqdm(glob.glob(f"{SLAMM_RESULTS}/*/*.geojson"))

for fpath in pbar:
    fpath = fpath.replace("\\", "/")
    pbar.set_description(f"Processing {fpath.split('/')[-1]}")
    match = FILE_PATH_PATTERN.search(fpath)
    if match is None:
        continue
    marsh_id = match.group(1)
    slr_scenario = match.group(2)
    year = match.group(3)
    marsh_initial = match.group(4)

    make_flood_map(
        fpath,
        inun_freq_key_df,
        flood_colors,
        order_leg,
        OUTPUT + f'{marsh_id}/{marsh_id}_{slr_scenario}slr_{year}_{marsh_initial}m_flood.png') 